### 本地数据读取和处理

In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import time

In [2]:
def read_img(file_path):
    start = time.time()
    file_name = os.listdir(file_path)
    x = []
    y = []
    for i in file_name:
        img = Image.open(file_path+i)
        img = img.resize((28,28)).convert('L')
        img_array = np.array(img)
        label = i.split('_')[-1].split('.')[0]
        x.append(img_array)
        y.append(label)
    x = np.array(x)
    y = np.array(y)
    y = y.astype(np.int64)
    print(f'run time is {time.time() - start}')
    return x,y

In [5]:
x_test,y_test = read_img("./mnist_image_label/mnist_train_jpg_60000/")

run time is 38.015801429748535


In [6]:
x_train,y_train = read_img("./mnist_image_label/mnist_train_jpg_60000/")

run time is 7.674140930175781


In [7]:
np.save("./x_test.npy",x_test)
np.save("./x_train.npy",x_train)

In [8]:
np.save("./y_test.npy",y_test)
np.save("./y_train.npy",y_train)

In [9]:
start = time.time()
x = np.load("./x_train.npy")
x = np.load("./y_train.npy")
print(f'run time is {time.time() - start}')
start2 = time.time()
x1 = np.load("./x_test.npy")
y1 = np.load("./y_test.npy")
print(f'run time is {time.time() - start2}')

run time is 0.031000137329101562
run time is 0.027999162673950195


# 数据增强

In [10]:
import tensorflow as tf
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.05,
    horizontal_flip=False,
    zoom_range=0.1
)

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),#扁平化 相当于reshape(num,-1)
    tf.keras.layers.Dense(200,activation="relu"),#relu,sigmoid
    tf.keras.layers.Dense(100,activation="relu"),
    tf.keras.layers.Dense(50,activation="relu"),
    tf.keras.layers.Dense(10,activation='softmax')#输出层,节点数=类别数，激活函数用softmax 概率总和为1
])

In [12]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics=['accuracy'])

checkpoint_save_path = "./checkpoint/mnist.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                save_weights_only=True,
                                                save_best_only=True)

model.fit(img_gen.flow(x_train.reshape(60000,28,28,1),y_train),batch_size=64,epochs=10,
          validation_data=(x_test,y_test),
         callbacks=[cp_callback])

model.summary()

Epoch 1/10
1875/1875 [==============================] - 9s 5ms/step - loss: 1.0905 - accuracy: 0.6668 - val_loss: 35.9355 - val_accuracy: 0.8888
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5382 - accuracy: 0.8407 - val_loss: 31.6495 - val_accuracy: 0.9156
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3787 - accuracy: 0.8861 - val_loss: 26.9833 - val_accuracy: 0.9362
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3027 - accuracy: 0.9097 - val_loss: 22.7761 - val_accuracy: 0.9454
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2602 - accuracy: 0.9218 - val_loss: 21.8639 - val_accuracy: 0.9498
Epoch 6/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2348 - accuracy: 0.9306 - val_loss: 17.4770 - val_accuracy: 0.9594
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2121 - accuracy: 0.9369 - val_loss: 18.1336 - val_ac